<a href="https://colab.research.google.com/github/anshulsolanki/stock_analysis/blob/main/stock-index_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install dependency if needed:
!pip install yfinance matplotlib

In [2]:
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go

In [37]:
import warnings

def do_comparison(index_symbol, stocks , index_name):
  warnings.filterwarnings("ignore", message="YF.download() has changed argument auto_adjust default to True", category=FutureWarning)

  # Download 5 years of data
  all_symbols = [index_symbol] + list(stocks.values())
  data = yf.download(all_symbols, period="5y")["Close"].ffill()

  # Time frames
  time_frames = {
      "1M": pd.DateOffset(months=1),
      "2M": pd.DateOffset(months=2),
      "3M": pd.DateOffset(months=3),
      "6M": pd.DateOffset(months=6),
      "1Y": pd.DateOffset(years=1),
      "3Y": pd.DateOffset(years=3),
      "5Y": pd.DateOffset(years=5),
  }

  # Create figure
  fig = go.Figure()

  # Add traces placeholders
  for stock in stocks.keys():
      fig.add_trace(go.Scatter(x=[], y=[], mode="lines", name=stock))

  # Add FMCG index as bold black line
  fig.add_trace(go.Scatter(x=[], y=[], mode="lines", name=index_name,
                          line=dict(color="black", width=3)))

  # Dropdown buttons
  buttons = []

  # Build performance table
  performance_table = pd.DataFrame(index=list(stocks.keys()) + [index_name])

  # Build RS table
  rs_table = pd.DataFrame(index=list(stocks.keys()))


  for label, offset in time_frames.items():
      today = data.index.max()
      start_date = today - offset
      df_slice = data[data.index >= start_date]

      # Normalize (start = 1)
      df_norm = df_slice / df_slice.iloc[0]

      # Collect data
      x_data, y_data = [], []

      performances ={}
      for stock, symbol in stocks.items():
          x_data.append(df_norm.index)
          y_data.append(df_norm[symbol])
          performances[stock] = df_norm[symbol].iloc[-1]  # final value

      # FMCG Index
      x_data.append(df_norm.index)
      y_data.append(df_norm[index_symbol])

      # Final normalized values for stocks
      performances = {stock: df_norm[symbol].iloc[-1] for stock, symbol in stocks.items()}

      # Add Nifty FMCG itself
      performances[index_name] = df_norm[index_symbol].iloc[-1]
      performance_table[label] = pd.Series(performances)
      performance_table = performance_table.round(3)

      # RS = Stock / Index
      rs_values = {
          stock: (df_norm[symbol].iloc[-1] / df_norm[index_symbol].iloc[-1])
          for stock, symbol in stocks.items()
      }

      rs_table[label] = pd.Series(rs_values)
      # Round for readability
      rs_table = rs_table.round(3)

      buttons.append(dict(
          label=label,
          method="update",
          args=[{"x": x_data, "y": y_data},
                {"title": f"Performance vs {index_name} ({label})"}]
      ))

  # Layout
  fig.update_layout(
      updatemenus=[dict(active=6, buttons=buttons, x=0.05, y=1.15, xanchor="left", yanchor="top")],
      title=f"Stock Performance vs {index_name}",
      xaxis_title="Date",
      yaxis_title="Normalized Performance",
      legend_title="Stocks"
  )

  fig.show()

  print("\n\n📊 Stock & Index Performance (End Value Normalized)")
  print(performance_table)
  return performance_table

  #print(f"📊 Relative Strength (Stock vs {index_name})")
  #print(rs_table)


In [34]:
def do_RS_based_analysis(index_symbol, stocks):
  # Download 5-year data
  all_symbols = [index_symbol] + list(stocks.values())
  data = yf.download(all_symbols, period="5y")["Close"].ffill()

  # Define time frames
  time_frames = {
      "1M": pd.DateOffset(months=1),
      "2M": pd.DateOffset(months=2),
      "3M": pd.DateOffset(months=3),
      "6M": pd.DateOffset(months=6),
      "1Y": pd.DateOffset(years=1),
      "3Y": pd.DateOffset(years=3),
      "5Y": pd.DateOffset(years=5)
  }

  # Build Relative Strength table
  rs_table = pd.DataFrame(index=stocks.keys())
  today = data.index.max()

  for label, offset in time_frames.items():
      start_date = today - offset
      df_slice = data[data.index >= start_date]
      df_norm = df_slice / df_slice.iloc[0]  # normalize
      rs_values = {stock: df_norm[symbol].iloc[-1] / df_norm[index_symbol].iloc[-1]
                  for stock, symbol in stocks.items()}
      rs_table[label] = pd.Series(rs_values)

  rs_table = rs_table.round(3)
  print("\n📊 Relative Strength Table:")
  print(rs_table)

  # ---- Momentum Analysis ----
  # Consistent long-term outperformers (6M & 1Y RS > 1)
  rs_table['Consistent'] = (rs_table['3M'] > 1) & (rs_table['6M'] > 1) & (rs_table['1Y'] > 1)

  # Emerging outperformers (RS trending up: 3M → 2M → 1M)
  rs_table['Emerging'] = (rs_table['1M'] > rs_table['2M']) & (rs_table['2M'] > rs_table['3M'])

  # Slowing stocks (RS trending down: 3M → 2M → 1M)
  rs_table['Slowing'] = (rs_table['1M'] < rs_table['2M']) & (rs_table['2M'] < rs_table['3M'])

  # Optional: Overall score (Emerging + Consistent + latest RS)
  rs_table['Score'] = 0
  rs_table.loc[rs_table['Consistent'], 'Score'] += 1
  rs_table.loc[rs_table['Emerging'], 'Score'] += 1
  rs_table['Score'] += rs_table['1M']  # latest RS as part of score

  # Sort by score
  rs_table_sorted = rs_table.sort_values(by='Score', ascending=False)

  print("\n\n📊 RS Analysis with Momentum Flags (Emerging / Slowing / Consistent)")
  print(rs_table_sorted[['Consistent','Emerging','Slowing','Score']])

In [35]:
import warnings

def do_RS_based_analysis_new(index_symbol, stocks, include_technical=False):
    # Suppress the specific FutureWarning
    warnings.filterwarnings("ignore", message="Series.__getitem__ treating keys as positions is deprecated", category=FutureWarning)
    warnings.filterwarnings("ignore", message="Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas", category=FutureWarning)
    warnings.filterwarnings("ignore", message="YF.download() has changed argument auto_adjust default to True", category=FutureWarning)

    # Download 5-year data
    all_symbols = [index_symbol] + list(stocks.values())
    data = yf.download(all_symbols, period="5y")["Close"].ffill()
    volume_data = None
    if include_technical:
        volume_data = yf.download(all_symbols, period="5y")["Volume"].ffill()

    # Define time frames
    time_frames = {
        "1M": pd.DateOffset(months=1),
        "2M": pd.DateOffset(months=2),
        "3M": pd.DateOffset(months=3),
        "6M": pd.DateOffset(months=6),
        "1Y": pd.DateOffset(years=1),
        "3Y": pd.DateOffset(years=3),
        "5Y": pd.DateOffset(years=5)
    }

    # Build Relative Strength table
    rs_table = pd.DataFrame(index=stocks.keys())
    today = data.index.max()

    for label, offset in time_frames.items():
        start_date = today - offset
        df_slice = data[data.index >= start_date]
        df_norm = df_slice / df_slice.iloc[0]  # normalize
        rs_values = {stock: df_norm[symbol].iloc[-1] / df_norm[index_symbol].iloc[-1]
                     for stock, symbol in stocks.items()}
        rs_table[label] = pd.Series(rs_values)

    rs_table = rs_table.round(3)
    #print("\n📊 Relative Strength Table:")
    #print(rs_table)

    # ---- Momentum Analysis ----
    # Consistent long-term outperformers (6M & 1Y RS > 1)
    rs_table['Consistent'] = (rs_table['3M'] > 1) & (rs_table['6M'] > 1) & (rs_table['1Y'] > 1)

    # Emerging short-term RS (1M → 2M → 3M)
    rs_table['Emerging'] = (rs_table['1M'] > rs_table['2M']) & (rs_table['2M'] > rs_table['3M'])

    # Slowing (RS trending down 1M → 2M → 3M)
    rs_table['Slowing'] = (rs_table['1M'] < rs_table['2M']) & (rs_table['2M'] < rs_table['3M'])

    # ---- Early Turnaround Signals ----
    # 1. Short-term RS rising (Emerging)
    short_term_rs_rising = rs_table['Emerging']

    # 2. Medium-term RS ≤ 1 (sector lagging)
    medium_term_lagging = (rs_table['6M'] <= 1) | (rs_table['1Y'] <= 1)

    # 3. Absolute performance improving (normalized return 3M → 6M)
    absolute_perf = pd.Series(index=stocks.keys(), dtype=float)
    for stock, symbol in stocks.items():
        perf_3M = data[symbol][-1] / data[symbol][-63] - 1  # approx 63 trading days ~ 3 months
        perf_6M = data[symbol][-1] / data[symbol][-126] - 1  # approx 126 trading days ~ 6 months
        absolute_perf[stock] = perf_3M > perf_6M  # True if performance improving

    rs_table['Absolute_per'] = absolute_perf
    rs_table['Early_Turnaround'] = short_term_rs_rising & medium_term_lagging & absolute_perf

    # ---- Optional: Technical levels & volume confirmation ----
    if include_technical:
        ma_breakout = pd.Series(index=stocks.keys(), dtype=bool)
        vol_surge = pd.Series(index=stocks.keys(), dtype=bool)
        for stock, symbol in stocks.items():
            close = data[symbol]
            ma50 = close.rolling(50).mean()
            ma200 = close.rolling(200).mean()
            ma_breakout[stock] = (close[-1] > ma50[-1]) & (close[-1] > ma200[-1])

            volume = volume_data[symbol]
            vol_avg = volume.rolling(20).mean()
            vol_surge[stock] = volume[-1] > 1.5 * vol_avg[-1]
            #print("\n vol_surge[", symbol,"]",volume[-1], vol_avg[-1],vol_surge[stock])
        rs_table['MA_Breakout'] = ma_breakout
        rs_table['Volume_Surge'] = vol_surge

        # Refine Early_Turnaround with technical filters
        #rs_table['Early_Turnaround'] = rs_table['Early_Turnaround'] & ma_breakout & vol_surge

    # Optional: Overall Score
    rs_table['Score'] = 0
    rs_table.loc[rs_table['Consistent'], 'Score'] += 1
    rs_table.loc[rs_table['Emerging'], 'Score'] += 1
    rs_table.loc[rs_table['Early_Turnaround'], 'Score'] += 2
    rs_table['Score'] += rs_table['1M']  # add latest RS as part of score

    # Sort by Score
    rs_table_sorted = rs_table.sort_values(by='Score', ascending=False)

    #print("\n📊 RS Analysis with Momentum Flags & Early Turnaround Signals")
    display_columns = ['Consistent','Emerging','Slowing','Absolute_per','Early_Turnaround']
    if include_technical:
        display_columns += ['MA_Breakout','Volume_Surge']
    display_columns += ['Score']
    #print(rs_table_sorted[display_columns])
    return rs_table_sorted


# **Index Comparison**

In [38]:
# Index and stocks
index_symbol = "^NSEI"  # Nifty 50

stocks = {
  "Nifty 50": "^NSEI",
  "Bank Nifty": "^NSEBANK",
  "Nifty IT": "^CNXIT",
  "Nifty Pharma": "^CNXPHARMA",
  "Nifty FMCG": "^CNXFMCG",
  "Nifty Auto": "^CNXAUTO",
  "Nifty Metal": "^CNXMETAL",
  "Nifty Realty": "^CNXREALTY",
  "Nifty Infra": "^CNXINFRA"
}
do_comparison(index_symbol, stocks,"Nifty 50" )
#do_RS_based_analysis(index_symbol, stocks )
do_RS_based_analysis_new(index_symbol, stocks , True )

/tmp/ipython-input-2648652631.py:8: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  9 of 9 completed


/tmp/ipython-input-3568357825.py:11: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[                       0%                       ]



📊 Stock & Index Performance (End Value Normalized)
                 1M     2M     3M     6M     1Y     3Y     5Y
Nifty 50      1.004  0.972  1.000  1.108  0.984  1.401  2.179
Bank Nifty    0.977  0.950  0.970  1.116  1.051  1.359  2.358
Nifty IT      0.989  0.891  0.933  0.910  0.812  1.244  1.905
Nifty Pharma  0.998  0.980  1.013  1.089  0.947  1.745  1.945
Nifty FMCG    1.010  1.011  1.012  1.098  0.884  1.287  1.816
Nifty Auto    1.102  1.099  1.129  1.266  1.019  1.987  3.308
Nifty Metal   1.037  1.017  1.049  1.115  1.051  1.617  3.952
Nifty Realty  0.961  0.897  0.879  1.054  0.837  1.850  4.002
Nifty Infra   0.990  0.947  0.993  1.139  0.961  1.751  2.838
Nifty 50      1.004  0.972  1.000  1.108  0.984  1.401  2.179


[*********************100%***********************]  9 of 9 completed
/tmp/ipython-input-3568357825.py:14: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  9 of 9 completed


,1M,2M,3M,6M,1Y,3Y,5Y,Consistent,Emerging,Slowing,Absolute_per,Early_Turnaround,MA_Breakout,Volume_Surge,Score
Nifty Auto,1.098,1.131,1.130,1.143,1.035,1.419,1.518,True,False,False,False,False,True,False,2.098
Nifty Metal,1.033,1.046,1.049,1.006,1.068,1.155,1.814,True,False,True,False,False,True,False,2.033
Nifty Realty,0.958,0.923,0.879,0.952,0.851,1.321,1.837,False,True,False,False,False,False,False,1.958
Nifty FMCG,1.006,1.041,1.012,0.991,0.898,0.919,0.833,False,False,False,False,False,True,False,1.006
Nifty 50,1.000,1.000,1.000,1.000,1.000,1.000,1.000,False,False,False,False,False,False,False,1.000
Nifty Pharma,0.994,1.009,1.013,0.983,0.962,1.246,0.893,False,False,True,False,False,False,False,0.994
Nifty Infra,0.987,0.975,0.993,1.028,0.977,1.250,1.302,False,False,False,False,False,False,False,0.987
Nifty IT,0.985,0.917,0.934,0.821,0.825,0.888,0.875,False,False,False,False,False,False,False,0.985
Bank Nifty,0.974,0.978,0.971,1.008,1.068,0.971,1.082,False,False,False,False,False,False,False,0.974


In [53]:
import warnings
import yfinance as yf
import pandas as pd

def do_RS_based_analysis_as_of_date(index_symbol, stocks, include_technical=False, as_of_date=None):
    # Suppress warnings
    warnings.filterwarnings("ignore", category=FutureWarning)

    # Download 5-year data
    all_symbols = [index_symbol] + list(stocks.values())
    data = yf.download(all_symbols, period="5y")["Close"].ffill()
    volume_data = None
    if include_technical:
        volume_data = yf.download(all_symbols, period="5y")["Volume"].ffill()

    # Trim data if backtesting
    if as_of_date is not None:
        data = data[data.index <= pd.to_datetime(as_of_date)]
        if include_technical:
            volume_data = volume_data[volume_data.index <= pd.to_datetime(as_of_date)]

    # Define time frames
    time_frames = {
        "1M": pd.DateOffset(months=1),
        "2M": pd.DateOffset(months=2),
        "3M": pd.DateOffset(months=3),
        "6M": pd.DateOffset(months=6),
        "1Y": pd.DateOffset(years=1),
        "3Y": pd.DateOffset(years=3),
        "5Y": pd.DateOffset(years=5)
    }

    # Build Relative Strength table
    rs_table = pd.DataFrame(index=stocks.keys())
    today = data.index.max()

    for label, offset in time_frames.items():
        start_date = today - offset
        df_slice = data[data.index >= start_date]
        df_norm = df_slice / df_slice.iloc[0]  # normalize
        rs_values = {stock: df_norm[symbol].iloc[-1] / df_norm[index_symbol].iloc[-1]
                     for stock, symbol in stocks.items()}
        rs_table[label] = pd.Series(rs_values)

    rs_table = rs_table.round(3)

    # ---- Momentum Analysis ----
    rs_table['Consistent'] = (rs_table['3M'] > 1) & (rs_table['6M'] > 1) & (rs_table['1Y'] > 1)
    rs_table['Emerging'] = (rs_table['1M'] > rs_table['2M']) & (rs_table['2M'] > rs_table['3M'])
    rs_table['Slowing'] = (rs_table['1M'] < rs_table['2M']) & (rs_table['2M'] < rs_table['3M'])

    # ---- Early Turnaround Signals ----
    short_term_rs_rising = rs_table['Emerging']
    medium_term_lagging = (rs_table['6M'] <= 1) | (rs_table['1Y'] <= 1)

    absolute_perf = pd.Series(index=stocks.keys(), dtype=float)
    for stock, symbol in stocks.items():
        try:
            perf_3M = data[symbol][-1] / data[symbol][-63] - 1
            perf_6M = data[symbol][-1] / data[symbol][-126] - 1
            absolute_perf[stock] = perf_3M > perf_6M
        except Exception:
            absolute_perf[stock] = False

    rs_table['Absolute_per'] = absolute_perf
    rs_table['Early_Turnaround'] = short_term_rs_rising & medium_term_lagging & absolute_perf

    # ---- Optional: Technical filters ----
    if include_technical:
        ma_breakout = pd.Series(index=stocks.keys(), dtype=bool)
        vol_surge = pd.Series(index=stocks.keys(), dtype=bool)
        for stock, symbol in stocks.items():
            close = data[symbol]
            ma50 = close.rolling(50).mean()
            ma200 = close.rolling(200).mean()
            ma_breakout[stock] = (close[-1] > ma50[-1]) & (close[-1] > ma200[-1])

            volume = volume_data[symbol]
            vol_avg = volume.rolling(20).mean()
            vol_surge[stock] = volume[-1] > 1.5 * vol_avg[-1]
        rs_table['MA_Breakout'] = ma_breakout
        rs_table['Volume_Surge'] = vol_surge

    # ---- Score ----
    rs_table['Score'] = 0
    rs_table.loc[rs_table['Consistent'], 'Score'] += 1
    rs_table.loc[rs_table['Emerging'], 'Score'] += 1
    rs_table.loc[rs_table['Early_Turnaround'], 'Score'] += 2
    rs_table['Score'] += rs_table['1M'] # add latest RS as part of score

    rs_table_sorted = rs_table.sort_values(by='Score', ascending=False)

    #print("\n📊 RS Analysis with Momentum Flags & Early Turnaround Signals")
    display_columns = ['Consistent','Emerging','Slowing','Absolute_per','Early_Turnaround']
    if include_technical:
        display_columns += ['MA_Breakout','Volume_Surge']
    display_columns += ['Score']
    print(rs_table_sorted[display_columns])
    return rs_table_sorted


def forward_test(index_symbol, stocks, as_of_date, horizon="1M", top_n=3):
    # Generate signals as of date
    signals = do_RS_based_analysis_new(index_symbol, stocks, as_of_date=as_of_date)
    top_picks = signals.head(top_n).index.tolist()

    # Download future data
    start_date = pd.to_datetime(as_of_date)
    if horizon == "1M":
        end_date = start_date + pd.DateOffset(months=1)
    elif horizon == "3M":
        end_date = start_date + pd.DateOffset(months=3)
    elif horizon == "6M":
        end_date = start_date + pd.DateOffset(months=6)
    else:
        raise ValueError("Horizon must be one of: '1M', '3M', '6M'")

    all_symbols = [index_symbol] + list(stocks.values())
    future_data = yf.download(all_symbols, start=start_date, end=end_date)["Close"].ffill()

    # Compute forward returns for ALL stocks
    all_results = {}
    for stock, symbol in stocks.items():
        try:
            entry = future_data[symbol].iloc[0]
            exit = future_data[symbol].iloc[-1]
            all_results[stock] = round((exit / entry - 1) * 100, 2)
        except Exception:
            all_results[stock] = None

    # Index return
    idx_entry = future_data[index_symbol].iloc[0]
    idx_exit = future_data[index_symbol].iloc[-1]
    index_ret = round((idx_exit / idx_entry - 1) * 100, 2)

    # Extract performance of top picks
    results = {stock: all_results[stock] for stock in top_picks}

    # Identify true top N performers by forward returns
    top_actual = sorted(all_results.items(), key=lambda x: (x[1] is not None, x[1]), reverse=True)[:top_n]

    # 📊 Print formatted report
    print(f"\n\n📆 Backtest Date: {as_of_date}")
    print(f"Top Picks (Signal-based): {top_picks}")
    print(f"Next {horizon} Returns (Top Picks):")

    avg_stock_ret = 0
    count = 0
    for stock, ret in results.items():
        if ret is not None:
            sign = "+" if ret >= 0 else ""
            print(f"   {stock}: {sign}{ret}%")
            avg_stock_ret += ret
            count += 1

    avg_stock_ret = avg_stock_ret / count if count > 0 else 0
    sign_idx = "+" if index_ret >= 0 else ""
    print(f"{index_symbol}: {sign_idx}{index_ret}%")

    # Print true best performers
    print(f"\n🏆 True Top {top_n} Performers (by actual forward returns):")
    for stock, ret in top_actual:
        sign = "+" if ret >= 0 else ""
        print(f"   {stock}: {sign}{ret}%")

    # Compare strategy vs. index
    if avg_stock_ret > index_ret:
        print("\n➡ Strategy Outperformed Index")
    elif avg_stock_ret < index_ret:
        print("\n➡ Strategy Underperformed Index")
    else:
        print("\n➡ Strategy Matched Index")

#FMCG sector analysis

In [56]:
# Index and stocks
index_symbol = "^CNXFMCG"  # Nifty FMCG
stocks = {
    "Hindustan Unilever": "HINDUNILVR.NS",
    "ITC": "ITC.NS",
    "Nestle India": "NESTLEIND.NS",
    "Varun Beverages": "VBL.NS",
    "Britannia Industries": "BRITANNIA.NS",
    "Godrej Consumer Products": "GODREJCP.NS",
    "Tata Consumer Products": "TATACONSUM.NS",
    "United Spirits": "UNITDSPR.NS",
    "Marico": "MARICO.NS",
    "Dabur India": "DABUR.NS",
    "Patanjali Foods": "PATANJALI.NS",
    "Colgate-Palmolive (India)": "COLPAL.NS",
    "United Breweries": "UBL.NS",
    "Radico Khaitan": "RADICO.NS",
    "Emami": "EMAMILTD.NS"
}
do_comparison(index_symbol, stocks,"Nifty FMCG" )
#do_RS_based_analysis_1(index_symbol, stocks )
#do_RS_based_analysis_new(index_symbol, stocks , True )

date = "2025-08-01"
do_RS_based_analysis_as_of_date(index_symbol, stocks, True, date)
forward_test(index_symbol, stocks, date, horizon="1M", top_n=3)

[*********************100%***********************]  16 of 16 completed




📊 Stock & Index Performance (End Value Normalized)
                              1M     2M     3M     6M     1Y     3Y     5Y
Hindustan Unilever         1.039  1.093  1.120  1.226  0.945  1.064  1.317
ITC                        0.984  0.979  0.972  1.024  0.824  1.365  2.615
Nestle India               1.066  0.998  1.011  1.105  0.975  1.307  1.589
Varun Beverages            0.924  1.013  1.004  0.989  0.782  2.288  7.186
Britannia Industries       1.078  1.046  1.099  1.304  1.052  1.732  1.768
Godrej Consumer Products   1.014  0.975  1.018  1.252  0.859  1.423  1.952
Tata Consumer Products     1.007  0.973  0.964  1.129  0.909  1.313  2.044
United Spirits             0.991  0.955  0.820  1.003  0.891    NaN    NaN
Marico                     1.022  1.013  1.051  1.254  1.154  1.466  2.141
Dabur India                1.042  1.077  1.130  1.132  0.861  0.992  1.179
Patanjali Foods            0.977  1.078  1.065  1.018  0.931  1.382  3.048
Colgate-Palmolive (India)  1.086  0.983  0.988 

[*********************100%***********************]  16 of 16 completed
[*********************100%***********************]  16 of 16 completed
[                       0%                       ]

                           Consistent  Emerging  Slowing Absolute_per  \
Varun Beverages                 False      True    False         True   
Colgate-Palmolive (India)       False      True    False         True   
Patanjali Foods                 False      True    False         True   
Hindustan Unilever               True     False    False         True   
Radico Khaitan                   True     False     True        False   
Emami                           False      True    False        False   
Britannia Industries             True     False     True        False   
Marico                           True     False     True        False   
United Breweries                False      True    False        False   
Tata Consumer Products          False      True    False        False   
Dabur India                     False     False     True         True   
Godrej Consumer Products        False     False    False        False   
ITC                             False     False    

[*********************100%***********************]  16 of 16 completed
[*********************100%***********************]  16 of 16 completed



📆 Backtest Date: 2025-08-01
Top Picks (Signal-based): ['Varun Beverages', 'Colgate-Palmolive (India)', 'Patanjali Foods']
Next 1M Returns (Top Picks):
   Varun Beverages: -4.83%
   Colgate-Palmolive (India): +3.27%
   Patanjali Foods: -4.48%
^CNXFMCG: -0.1%

🏆 True Top 3 Performers (by actual forward returns):
   Hindustan Unilever: +4.15%
   Colgate-Palmolive (India): +3.27%
   Marico: +2.06%

➡ Strategy Underperformed Index


In [40]:
# Index and stocks
index_symbol = "^NSEBANK"  # Nifty FMCG
stocks = {
    "HDFC": "HDFCBANK.NS",
    "ICICI": "ICICIBANK.NS",
    "SBI": "SBIN.NS",
    "Kotak": "KOTAKBANK.NS",
    "Axis": "AXISBANK.NS",
    "BOB": "BANKBARODA.NS",
    "IDFC First": "IDFCFIRSTB.NS",
    "Federal Bank India": "FEDERALBNK.NS"
}
do_comparison(index_symbol, stocks,"Nifty Bank" )
do_RS_based_analysis_new(index_symbol, stocks )

/tmp/ipython-input-2648652631.py:8: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  9 of 9 completed


/tmp/ipython-input-3568357825.py:11: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[                       0%                       ]



📊 Stock & Index Performance (End Value Normalized)
                       1M     2M     3M     6M     1Y     3Y     5Y
HDFC                0.974  0.970  0.999  1.153  1.184  1.354  1.853
ICICI               0.979  0.985  0.972  1.163  1.144  1.629  3.884
SBI                 1.007  1.000  1.001  1.127  1.005  1.583  4.205
Kotak               0.971  0.906  0.955  1.008  1.095  1.007  1.427
Axis                0.986  0.899  0.912  1.042  0.895  1.400  2.312
BOB                 0.973  0.972  0.967  1.181  0.994  1.917  5.720
IDFC First          1.047  0.937  1.090  1.257  0.971  1.434  2.298
Federal Bank India  0.978  0.891  0.926  1.056  1.011  1.572  3.771
Nifty Bank          0.977  0.950  0.970  1.116  1.051  1.359  2.358


[*********************100%***********************]  9 of 9 completed


,1M,2M,3M,6M,1Y,3Y,5Y,Consistent,Emerging,Slowing,Absolute_per,Early_Turnaround,Score
Axis,1.009,0.946,0.940,0.933,0.851,1.030,0.980,False,True,False,False,False,2.009
ICICI,1.001,1.036,1.001,1.042,1.088,1.198,1.647,True,False,False,False,False,2.001
HDFC,0.997,1.020,1.030,1.033,1.126,0.996,0.786,True,False,True,False,False,1.997
IDFC First,1.071,0.986,1.124,1.126,0.923,1.055,0.974,False,False,False,False,False,1.071
SBI,1.031,1.052,1.031,1.009,0.956,1.164,1.783,False,False,False,False,False,1.031
Federal Bank India,1.000,0.938,0.954,0.946,0.961,1.157,1.599,False,False,False,False,False,1.000
BOB,0.996,1.023,0.996,1.058,0.945,1.410,2.425,False,False,False,False,False,0.996
Kotak,0.994,0.953,0.984,0.903,1.042,0.741,0.605,False,False,False,False,False,0.994


In [ ]:
index_symbol = "^CNXAUTO"  # Nifty Auto

# 15 constituent stocks of the Nifty Auto index
stocks = {
    "Maruti Suzuki India": "MARUTI.NS",
    "Mahindra & Mahindra": "M&M.NS",
    "Tata Motors": "TATAMOTORS.NS",
    "Bajaj Auto": "BAJAJ-AUTO.NS",
    "Eicher Motors": "EICHERMOT.NS",
    "TVS Motor Company": "TVSMOTOR.NS",
    "Bosch": "BOSCHLTD.NS",
    "Hero MotoCorp": "HEROMOTOCO.NS",
    "Samvardhana Motherson": "MOTHERSON.NS",
    "Ashok Leyland": "ASHOKLEY.NS",
    "MRF": "MRF.NS",
    "Tube Investments of India": "TIINDIA.NS",
    "Bharat Forge": "BHARATFORG.NS",
    "Balkrishna Industries": "BALKRISIND.NS",
    "Exide Industries": "EXIDEIND.NS"
}

do_comparison(index_symbol, stocks,"Nifty auto" )
do_RS_based_analysis(index_symbol, stocks )

In [ ]:
#Nifty IT components
stocks = {
    "Tata Consultancy Services": "TCS.NS",
    "Infosys": "INFY.NS",
    "HCL Technologies": "HCLTECH.NS",
    "Wipro": "WIPRO.NS",
    "Tech Mahindra": "TECHM.NS",
    "LTIMindtree": "LTIM.NS",
    "Persistent Systems": "PERSISTENT.NS",
    "Oracle Financial Services Software": "OFSS.NS",
    "Coforge": "COFORGE.NS",
    "Mphasis": "MPHASIS.NS"
}
#Nifty FMCG components
stocks = {
    "Hindustan Unilever": "HINDUNILVR.NS",
    "ITC": "ITC.NS",
    "Nestle India": "NESTLEIND.NS",
    "Varun Beverages": "VBL.NS",
    "Britannia Industries": "BRITANNIA.NS",
    "Godrej Consumer Products": "GODREJCP.NS",
    "Tata Consumer Products": "TATACONSUM.NS",
    "United Spirits": "MCDOWELL-N.NS",  # Yahoo puts it as "MCDOWELL-N"
    "Marico": "MARICO.NS",
    "Dabur India": "DABUR.NS",
    "Patanjali Foods": "PATANJALI.NS",
    "Colgate-Palmolive (India)": "COLPAL.NS",
    "United Breweries": "UBL.NS",
    "Radico Khaitan": "RADICO.NS",
    "Emami": "EMAMILTD.NS"
}

#Nifty Pharma components
stocks = {
    "Sun Pharmaceutical Industries": "SUNPHARMA.NS",
    "Divi's Laboratories": "DIVISLAB.NS",
    "Cipla": "CIPLA.NS",
    "Torrent Pharmaceuticals": "TORNTPHARM.NS",
    "Dr. Reddy's Laboratories": "DRREDDY.NS",
    "Mankind Pharma": "MANKINDP.NS",
    "Zydus Lifesciences": "ZYDUSLIFE.NS",
    "Lupin": "LUPIN.NS",
    "Abbott India": "ABBOTINDIA.NS",
    "Alkem Laboratories": "ALKEM.NS",
    "Aurobindo Pharma": "AUROPHARMA.NS",
    "Glenmark Pharmaceuticals": "GLENMARK.NS",
    "Biocon": "BIOCON.NS",
    "Laurus Labs": "LAURUSLABS.NS",
    "IPCA Laboratories": "IPCALAB.NS",
    "Gland Pharma": "GLAND.NS",
    "Ajanta Pharma": "AJANTPHARM.NS",
    "JB Chemicals & Pharmaceuticals": "JBCHEPHARM.NS",
    "Natco Pharma": "NATCOPHARM.NS",
    "Granules India": "GRANULES.NS"
}

#Nifty Auto components
stocks = {
    "Maruti Suzuki India": "MARUTI.NS",
    "Mahindra & Mahindra": "M&M.NS",
    "Tata Motors": "TATAMOTORS.NS",
    "Bajaj Auto": "BAJAJ-AUTO.NS",
    "Eicher Motors": "EICHERMOT.NS",
    "TVS Motor Company": "TVSMOTOR.NS",
    "Hero MotoCorp": "HEROMOTOCO.NS",
    "Samvardhana Motherson": "MOTHERSUMI.NS",
    "Ashok Leyland": "ASHOKLEY.NS",
    "Bharat Forge": "BHARATFORG.NS",
    "Exide Industries": "EXIDEIND.NS",
    "MRF": "MRF.NS",
    "Tube Investments of India": "TUBEINVEST.NS",
    "Balkrishna Industries": "BALKRISIND.NS",
    "Bosch": "BOSCHLTD.NS"
}

#Nifty Bank components
stocks = {
    "HDFC Bank": "HDFCBANK.NS",
    "ICICI Bank": "ICICIBANK.NS",
    "State Bank of India": "SBIN.NS",
    "Kotak Mahindra Bank": "KOTAKBANK.NS",
    "Axis Bank": "AXISBANK.NS",
    "Bank of Baroda": "BANKBARODA.NS",
    "Punjab National Bank": "PNB.NS",
    "IndusInd Bank": "INDUSINDBK.NS",
    "IDFC First Bank": "IDFCFIRSTB.NS",
    "AU Small Finance Bank": "AUBANK.NS",
    "Federal Bank": "FEDERALBNK.NS",
    "Bandhan Bank": "BANDHANBNK.NS"
}

#Nifty Metal components
stocks = {
    "Hindalco Industries": "HINDALCO.NS",
    "JSW Steel": "JSWSTEEL.NS",
    "Tata Steel": "TATASTEEL.NS",
    "Steel Authority of India": "SAIL.NS",
    "National Aluminium Company": "NALCO.NS",
    "Vedanta": "VEDL.NS",
    "Jindal Steel & Power": "JINDALSTEL.NS",
    "Hindustan Zinc": "HZL.NS",
    "NMDC": "NMDC.NS",
    "Apl Apollo Tubes": "APLAPOLLO.NS",
    "Ratnamani Metals & Tubes": "RATNAMANI.NS",
    "Welspun Corp": "WELCORP.NS"
}

#Nifty Metal components
stocks = {
    "Indian Oil Corporation": "IOC.NS",
    "Larsen & Toubro": "LT.NS",
    "Max Healthcare Institute": "MAXHEALTH.NS",
    "Samvardhana Motherson International": "MOTHERSUMI.NS",
    "NTPC": "NTPC.NS",
    "Oil & Natural Gas Corporation": "ONGC.NS",
    "Power Grid Corporation of India": "POWERGRID.NS",
    "Reliance Industries": "RELIANCE.NS",
    "Shree Cement": "SHREECEM.NS",
    "Siemens": "SIEMENS.NS",
    "Tata Power": "TATAPOWER.NS",
    "UltraTech Cement": "ULTRACEMCO.NS"
}

#nifty 50 components
{
  "HDFC Bank": "HDFCBANK.NS",
  "ICICI Bank": "ICICIBANK.NS",
  "State Bank of India": "SBIN.NS",
  "Kotak Mahindra Bank": "KOTAKBANK.NS",
  "Axis Bank": "AXISBANK.NS",
  "Bajaj Finance": "BAJFINANCE.NS",
  "Bajaj Auto": "BAJAJ-AUTO.NS",
  "Bharti Airtel": "BHARTIARTL.NS",
  "HCL Technologies": "HCLTECH.NS",
  "HDFC Life Insurance": "HDFCLIFE.NS",
  "Hero MotoCorp": "HEROMOTOCO.NS",
  "Hindalco Industries": "HINDALCO.NS",
  "Hindustan Unilever": "HINDUNILVR.NS",
  "ICICI Prudential Life Insurance": "ICICIPRULI.NS",
  "IndusInd Bank": "INDUSINDBK.NS",
  "Infosys": "INFY.NS",
  "ITC": "ITC.NS",
  "JSW Steel": "JSWSTEEL.NS",
  "Larsen & Toubro": "LT.NS",
  "Mahindra & Mahindra": "M&M.NS",
  "Maruti Suzuki": "MARUTI.NS",
  "NTPC": "NTPC.NS",
  "Oil & Natural Gas Corporation": "ONGC.NS",
  "Power Grid Corporation": "POWERGRID.NS",
  "Reliance Industries": "RELIANCE.NS",
  "Shree Cement": "SHREECEM.NS",
  "Sun Pharmaceutical Industries": "SUNPHARMA.NS",
  "Tata Consumer Products": "TATACONSUM.NS",
  "Tata Motors": "TATAMOTORS.NS",
  "Tata Consultancy Services": "TCS.NS",
  "Tech Mahindra": "TECHM.NS",
  "Titan Company": "TITAN.NS",
  "UltraTech Cement": "ULTRACEMCO.NS",
  "UPL": "UPL.NS",
  "Wipro": "WIPRO.NS",
  "Adani Enterprises": "ADANIENT.NS",
  "Adani Ports and Special Economic Zone": "ADANIPORTS.NS",
  "Apollo Hospitals Enterprise": "APOLLOHOSP.NS",
  "Asian Paints": "ASIANPAINT.NS",
  "Bajaj Finserv": "BAJAJFINSV.NS",
  "Bharat Electronics": "BEL.NS",
  "Coal India": "COALINDIA.NS",
  "Eicher Motors": "EICHERMOT.NS",
  "Grasim Industries": "GRASIM.NS",
  "Indiabulls Housing Finance": "IBULHSGFIN.NS",
  "Jio Financial Services": "JIOFIN.NS",
  "Lupin": "LUPIN.NS",
  "Motherson Sumi Wiring India": "MOTHERSUMI.NS",
  "Pidilite Industries": "PIDILITE.NS",
  "SBI Life Insurance": "SBILIFE.NS",
  "SBI Cards and Payment Services": "SBICARD.NS",
  "Shree Cement": "SHREECEM.NS",
  "Sun Pharmaceutical Industries": "SUNPHARMA.NS",
  "Tata Steel": "TATASTEEL.NS",
  "Tech Mahindra": "TECHM.NS",
  "Titan Company": "TITAN.NS",
  "UltraTech Cement": "ULTRACEMCO.NS",
  "UPL": "UPL.NS",
  "Wipro": "WIPRO.NS"
}


